<a href="https://colab.research.google.com/github/Hasnainghori112/context-agent-openaiSDK/blob/main/context_openai_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


In [12]:
import os
from agents import Agent, Runner,  ItemHelpers,AsyncOpenAI, OpenAIChatCompletionsModel , function_tool , RunContextWrapper
from agents.run import RunConfig
from google.colab import userdata
from dataclasses import dataclass
import asyncio
from agents import (
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,
)

gemini_api_key = userdata.get("GOOGLE_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

set_default_openai_client(client=external_client, use_for_tracing=False)
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)

In [51]:
from openai.types.responses import ResponseTextDeltaEvent
@dataclass
class UserInfo():
  name : str
  age : int
  gender : str
  country : str


@function_tool
async def fatch_user_name(wrapper : RunContextWrapper):
  return f"The user name is {wrapper.context.name}"

@function_tool
async def fatch_user_age(wrapper : RunContextWrapper):
  return f"The user age is {wrapper.context.age}"
@function_tool
async def fatch_user_gender(wrapper : RunContextWrapper):
  return f"The user gender is {wrapper.context.gender}"

@function_tool
async def fatch_user_country(wrapper : RunContextWrapper):
  return f"The user country is {wrapper.context.country}"

user_info = UserInfo(
    name = "Muhammad Hasnain Ghori",
    age = 17,
    gender = "Male",
    country = "Pakistan"
)
assistan = Agent[UserInfo](
    name = "assistant",
    instructions ="""You are a helpful assistant. Your job is to gather all user details one by one by calling appropriate tools.
    Use only one tool at a time,with catchy style""",
    model=model,
    tools=[
        fatch_user_name,
        fatch_user_age,
        fatch_user_gender,
        fatch_user_country
    ],

)


async def main():
  result = Runner.run_streamed(assistan , "please provide all about user info",context=user_info)
  async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data,ResponseTextDeltaEvent):
      print(event.data.delta , end="" , flush=True)

asyncio.run(main())

Alright, let's start gathering user information. First, I'll need the user's name.
Okay, I've got the user's name. It's Muhammad Hasnain Ghori. Now, let's move on to the next piece of information. How about the user's age?
Great! The user is 17 years old. Now that we have the name and age, let's find out the user's gender.
Got it! The user is male. Just one more piece of information to complete the profile. Let's get the user's country.
Excellent! I have gathered all the user information. The user's name is Muhammad Hasnain Ghori, he is 17 years old, male, and from Pakistan.


In [94]:
from openai.types.responses import ResponseTextDeltaEvent
@dataclass
class UserInfo():
  name : str
  age : int
  gender : str
  country : str


@function_tool
async def fatch_user_name(wrapper : RunContextWrapper):
  return f"The user name is {wrapper.context.name}"

@function_tool
async def fatch_user_age(wrapper : RunContextWrapper):
  return f"The user age is {wrapper.context.age}"
@function_tool
async def fatch_user_gender(wrapper : RunContextWrapper):
  return f"The user gender is {wrapper.context.gender}"

@function_tool
async def fatch_user_country(wrapper : RunContextWrapper):
  return f"The user country is {wrapper.context.country}"

user_info = UserInfo(
    name = "MUhammad Hasnain Ghori",
    age = 17,
    gender = "Male",
    country = "Pakistan"
)
assistan = Agent[UserInfo](
    name = "assistant",
    instructions ="""You are a helpful assistant. Your job is to gather all user details one by one by calling appropriate tools.
    Use only one tool at a time,with catchy style""",
    model=model,
    tools=[
        fatch_user_name,
        fatch_user_age,
        fatch_user_gender,
        fatch_user_country
    ],

)


async def main():
  result = Runner.run_streamed(assistan , "please provide all about user info",context=user_info)
  async for event in result.stream_events():
    # print(event)
    if event.type == "raw_response_event":
      continue
    elif event.type == "agent_updated_stream_event":
      print(f"agent name : {event.new_agent.name}")
      continue
    elif event.type == "run_item_stream_event":
      if event.item.type == "tool_call_item":
        tool_name = event.item.raw_item.name
        print(f"Tool was called: {tool_name}")
      elif event.item.type == "tool_call_output_item":
        print(f"TOOl OUTPUT = {event.item.output}")
      elif event.item.type == "message_output_item":
         print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}" )
      else:
        pass

asyncio.run(main())

agent name : assistant
-- Message output:
 Alright, let's start gathering user details one piece at a time. To begin, I'll need to know the user's name.


-- Tool was called: fatch_user_name
TOOl OUTPUT = The user name is John
-- Message output:
 Fantastic! The user's name is John. Next up, let's find out the user's age.


-- Tool was called: fatch_user_age
TOOl OUTPUT = The user age is 25
-- Message output:
 Great! We know John is 25 years old. Now, let's discover John's gender.


-- Tool was called: fatch_user_gender
TOOl OUTPUT = The user gender is Male
-- Message output:
 Roger that! John is a male. Just one more detail to uncover: John's country.


-- Tool was called: fatch_user_country
TOOl OUTPUT = The user country is USA
-- Message output:
 Perfect! We've gathered all the info. The user, John, is a 25-year-old male from the USA.

